In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import casadi as cas
import do_mpc

from cas_models.continuous_time.models import StateSpaceModelCT
from cas_models.transformations import connect_systems
from feed_conc_ctrl.models import MixingTankModelCT, FlowMixerCT

## Construct Mixing Tank System Model using `cas_models`

In [ ]:
D = 5  # tank diameter [m]
n_tanks = 2
tank_names = [f"tank_{i+1}" for i in range(n_tanks)]

# Initialize tank system models
systems = [MixingTankModelCT(D=D, name=name) for name in tank_names]

# Add a flow mixer to join flows from two tanks
systems.append(FlowMixerCT(2, name="mixer"))

def print_sys_dimensions(sys):
    print(sys.name, f"({sys.ny}x{sys.nu})")
    for attr_name in ["input_names", "state_names", "output_names"]:
        print(f"{attr_name:>15s}: {getattr(sys, attr_name)}")

# Check inputs and outputs
for sys in systems:
    print_sys_dimensions(sys)

In [ ]:
# Connect all systems together
connections = {
    'mixer_conc_in_1': 'tank_1_conc_out',
    'mixer_conc_in_2': 'tank_2_conc_out',
    'mixer_v_dot_in_1': 'tank_1_v_dot_out',
    'mixer_v_dot_in_2': 'tank_2_v_dot_out'
}

model_class = StateSpaceModelCT
feed_tanks_system = connect_systems(
    systems, connections, model_class, name="tank_system_21", verbose_names=True
) 
print_sys_dimensions(feed_tanks_system)

## Construct DO-MPC System Model

States
 1. Tank 1 level
 2. Tank 1 mass
 3. Tank 2 level
 4. Tank 2 mass

Controlled Variables (CVs):
 1. Tank 1 level
 2. Tank 1 outflow concentration
 3. Tank 2 level
 4. Tank 2 outflow concentration
 5. Mixer outflow concentration

Manipulated Variables (MVs):
 1. Tank 1 inflow rate
 2. Tank 2 inflow rate
 3. Mixer inflow ratio (tank 1 flow rate / total flow rate)

Unmeasured Disturbances
 1. Tank 1 inflow concentration
 2. Tank 2 inflow concentration
 3. Mixer outflow rate

In [ ]:
model_type = 'continuous' # either 'discrete' or 'continuous'
model = do_mpc.model.Model(model_type)

# Define state variables
for tank_name in tank_names:
    model.set_variable(var_type='_x', var_name=tank_name + '_L', shape=(1, 1))
    model.set_variable(var_type='_x', var_name=tank_name + '_m', shape=(1, 1))
n = model.x.shape[0]

# Augment model with states for unmeasured disturbances
for tank_name in tank_names:
    model.set_variable(var_type='_x', var_name=tank_name + '_conc_in', shape=(1, 1))
model.set_variable(var_type='_x', var_name='mixer_v_dot_out', shape=(1, 1))
n_aug = model.x.shape[0]

# Define manipulated input variables
for tank_name in tank_names:
    model.set_variable(var_type='_u', var_name=tank_name + '_v_dot_in', shape=(1, 1))
model.set_variable(var_type='_u', var_name='mixer_ratio', shape=(1, 1))

print("States: ", list(model.x.keys()))
print("MVs: ", list(model.u.keys()))

# Define experessions for tank outflow rates based on mixer ratio 
tank_1_v_dot_out = model.u['mixer_ratio'] * model.x['mixer_v_dot_out']
tank_2_v_dot_out = (1.0 - model.u['mixer_ratio']) * model.x['mixer_v_dot_out']

t = 0.0  # not used in this model
states = cas.vcat(model.x[feed_tanks_system.state_names])
inputs = cas.vcat([
    model.u['tank_1_v_dot_in'], 
    model.x['tank_1_conc_in'], 
    tank_1_v_dot_out, 
    model.u['tank_2_v_dot_in'], 
    model.x['tank_2_conc_in'], 
    tank_2_v_dot_out
])
assert inputs.shape == (feed_tanks_system.nu, 1)
outputs = feed_tanks_system.h(t, states, inputs)
output_expressions = {
    name: outputs[i] for i, name in enumerate(feed_tanks_system.output_names)
}

# Define measured variables and output expressions
for tank_name in tank_names:
    meas_name = tank_name + '_L'
    model.set_meas(
        meas_name=meas_name, expr=output_expressions[meas_name]
    )
    meas_name = tank_name + '_conc_out'
    model.set_meas(
        meas_name=meas_name, expr=output_expressions[meas_name]
    )
meas_name = 'mixer_conc_out'
model.set_meas(
    meas_name=meas_name, expr=output_expressions[meas_name]
)
print("CVs: ", list(model.y.keys()))